In [470]:
import pandas as pd
from openai import OpenAI
import json
import tqdm
import matplotlib.pyplot as plt
import datetime
import concurrent.futures
from collections import Counter
import copy

client = OpenAI(
    # This is the default and can be omitted
    api_key="sk-dP3OLsFmsPb2kgfFaeYO5im3fx9KZkLtVBsMwRqMwvzsdQtI",
    base_url="https://api.chatanywhere.tech/v1"
)
secret_word_used = set(["Leonardo DiCaprio", "Taylor Swift", "Beyoncé"])
topic = "animals"

In [471]:
system_prompt = f"""
Now we will play a game called Chameleon. I want you to be the game organizer.
Your task is to update and process game information.
Imagine we have 4 players. You should randomly assign a role to each player. Which means
there should be one Chameleon among all players.
Each player is controlled by real person. I will update their choices after each
round. You should not say any unrelated words. Now let's begin. You should randomly give a topic and secret word.
The format should be in a json foramt:
{{"Topic": {topic},  
"Secret Word": "",
"Roles":{{"player1": "non-chameleon", "player2":"non-chameleon", "player3":"chameleon", "player4":"non-chameleon"}}
}}
You should assign a secret word based on topic {topic}, don't use the word in {secret_word_used}. Also the roles should be randomized.
Don't let player1 be chameleon.
"""

In [472]:
print(system_prompt)


Now we will play a game called Chameleon. I want you to be the game organizer.
Your task is to update and process game information.
Imagine we have 4 players. You should randomly assign a role to each player. Which means
there should be one Chameleon among all players.
Each player is controlled by real person. I will update their choices after each
round. You should not say any unrelated words. Now let's begin. You should randomly give a topic and secret word.
The format should be in a json foramt:
{"Topic": animals,  
"Secret Word": "",
"Roles":{"player1": "non-chameleon", "player2":"non-chameleon", "player3":"chameleon", "player4":"non-chameleon"}
}
You should assign a secret word based on topic animals, don't use the word in {'Leonardo DiCaprio', 'Beyoncé', 'Taylor Swift'}. Also the roles should be randomized.
Don't let player1 be chameleon.



In [473]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        }
    ],
    model="gpt-4o-mini",
)
response = chat_completion.choices[0].message.content
print(response)

{
  "Topic": "animals",
  "Secret Word": "elephant",
  "Roles": {
    "player1": "non-chameleon",
    "player2": "chameleon",
    "player3": "non-chameleon",
    "player4": "non-chameleon"
  }
}


In [474]:
cleaned_string = response.replace('\n', '')
system_game_info = json.loads(response)
system_game_info

{'Topic': 'animals',
 'Secret Word': 'elephant',
 'Roles': {'player1': 'non-chameleon',
  'player2': 'chameleon',
  'player3': 'non-chameleon',
  'player4': 'non-chameleon'}}

In [475]:
# system_game_info['Secret Word'] = "Edward Norton"

In [476]:
agent_prompt_discussion_stage = """
Now we will play a game called Chameleon. I want you to be one of the players.
If you are a chameleon, try to guess the secret word.
If you are not a chameleon, talk about something related to the topic and secret word, but do not reveal the secret word directly.
No matter the role you are, you should only say one word.
I will provide you the game info in a json format.
For example:
{'Topic': 'Fruits', 'Secret Word': 'Mango', 'Roles': 'non-chameleon', 'Round': 1, 'your position': 'player2', 'discussion': {'player1': ['Tropical'], 'player2': [], 'player3': [], 'player4': []}}
"""

agent_prompt_discussion_stage = """
Now we will play a game called Chameleon. I want you to be one of the players.
If you are a chameleon, try to guess the secret word in heart. But don't say it out loud. Because your goal is to hide your identity.
You can say something similar like what other people said.
If you are not a chameleon, talk about something related to the topic and secret word, but do not reveal the secret word directly.
Try to be vague, because if you are too obvious, the chameleon could easily have guessed it. But don't be too vague. your answer should be related to
topic and secret word. It's better to be specific but not revealing too much.
No matter each role, you should just say one short sentence. You shouldn't mention topic and secret word. 
And the sentence can be just few words. Don't be grammarly correct. Say it just like normal people do.
I will provide you the game info in a json format.
For example:
{'Topic': 'Fruits', 'Secret Word': 'Mango', 'Roles': 'non-chameleon', 'Round': 1, 'your position': 'player2', 'discussion': {'player1': ['Tropical'], 'player2': [], 'player3': [], 'player4': []}}
"""

In [477]:
discussion_history = {"player1": [], "player2":[], "player3":[], "player4":[]}
roles = system_game_info['Roles']
game_info_archive = {"round1": {"player1": [], "player2":[], "player3":[], "player4":[]}}
chameleon_player = [key for key, value in roles.items() if value == 'chameleon'][0]
secret_word_used.add(system_game_info['Secret Word'])

In [478]:
for player in discussion_history.keys():
    role = roles[player]
    if role == 'non-chameleon':
        game_info = {'Topic': system_game_info['Topic'], 'Secret Word': system_game_info['Secret Word'], 'Roles': role, "Round": 1,
                            "your position": player,
                            "discussion":discussion_history}
    else:
        game_info = {'Topic': '', 'Secret Word': '', 'Roles': role, "Round": 1,
                            "your position": player,
                            "discussion":discussion_history}
    print(game_info)
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": agent_prompt_discussion_stage,
            },
            {
                "role":"user",
                "content": f"game info{game_info}"
            }
        ],
        model="gpt-4o-mini",
    )
    response = chat_completion.choices[0].message.content
    print(response)
    discussion_history[player].append(response)
    game_info["discussion"] = copy.deepcopy(discussion_history)
    game_info_archive['round1'][player].append(game_info)

{'Topic': 'animals', 'Secret Word': 'elephant', 'Roles': 'non-chameleon', 'Round': 1, 'your position': 'player1', 'discussion': {'player1': [], 'player2': [], 'player3': [], 'player4': []}}
Big and gray.
{'Topic': '', 'Secret Word': '', 'Roles': 'chameleon', 'Round': 1, 'your position': 'player2', 'discussion': {'player1': ['Big and gray.'], 'player2': [], 'player3': [], 'player4': []}}
Huge and majestic.
{'Topic': 'animals', 'Secret Word': 'elephant', 'Roles': 'non-chameleon', 'Round': 1, 'your position': 'player3', 'discussion': {'player1': ['Big and gray.'], 'player2': ['Huge and majestic.'], 'player3': [], 'player4': []}}
They have long trunks.
{'Topic': 'animals', 'Secret Word': 'elephant', 'Roles': 'non-chameleon', 'Round': 1, 'your position': 'player4', 'discussion': {'player1': ['Big and gray.'], 'player2': ['Huge and majestic.'], 'player3': ['They have long trunks.'], 'player4': []}}
They’re known for their intelligence.


In [479]:
discussion_history

{'player1': ['Big and gray.'],
 'player2': ['Huge and majestic.'],
 'player3': ['They have long trunks.'],
 'player4': ['They’re known for their intelligence.']}

In [480]:
game_info_public = {"Round": 1, "discussion":discussion_history}
game_info_public

{'Round': 1,
 'discussion': {'player1': ['Big and gray.'],
  'player2': ['Huge and majestic.'],
  'player3': ['They have long trunks.'],
  'player4': ['They’re known for their intelligence.']}}

In [481]:
voting_prompt=f"""game info: {game_info_public}
Now the discussion stage on round 1 is over. Now it's the voting stage.
Each player should vote out the player whom you think is most like a chameleon.
give your reason, then
Vote a player by saying its number. You should state one number.
The format should be in a python array : [a single number you voted, your reasoning]
In your reasoning, state your role and player number first, then think. If you are chameleon, try to guess the secret word and topic.
"""

In [482]:
for player in discussion_history.keys():
    role = roles[player]
    game_info = game_info_archive['round1'][player]
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": agent_prompt_discussion_stage,
            },
            {
                "role" : "user",
                "content": f"game info for round1 discussion stage: {game_info}"
            },
            {
                "role" : "user",
                "content": voting_prompt
            }
        ],
        model="gpt-4o-mini",
    )
    response = chat_completion.choices[0].message.content
    print(response)
    

[3, "I'm a non-chameleon player1. Player3 seemed to focus too much on the trunk; it felt a bit specific compared to the others."]
[1, "I'm the chameleon, but I'm guessing the secret word might be related to elephants. Player 1's description seems off."]
[1, "I'm player3 and I'm a non-chameleon. Player1's description is pretty generic, could be applicable to many animals."]
[1, "I'm player4, a non-chameleon. Player1's description feels too generic, might not know the specific word."]


In [483]:
role = roles[chameleon_player]
game_info = game_info_archive['round1'][chameleon_player]
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": agent_prompt_discussion_stage,
        },
        {
            "role" : "user",
            "content": f"game info for round1 discussion stage: {game_info}"
        },
        {
            "role" : "user",
            "content": f"""game info after round1: {game_info_public} This is the final round for chameleon player, which is you.
            in order for you to win,
            first state your role and which player you are, then do step by step reasoning to guess the topic and secret word
            at this stage, you can safely say it out loud"""
        }
    ],
    model="gpt-4o-mini",
)
response = chat_completion.choices[0].message.content
print(response)

I am the chameleon, and I am player 2.

Step 1: Analyze the clues given by other players.
- Player 1 mentioned "Big and gray."
- Player 2 (myself) said "Huge and majestic."
- Player 3 added "They have long trunks."
- Player 4 said "They’re known for their intelligence."

Step 2: Identify common elements in the clues.
All the clues suggest a large animal, likely a mammal. The description of being "big and gray" along with "long trunks" points toward a specific type of animal.

Step 3: Consider the topic and the secret word based on these clues.
The presence of "long trunks" is a strong indicator that the topic is related to elephants.

Final Guess:
I believe the topic is "Animals," and the secret word is "Elephant."
